<a href="https://colab.research.google.com/github/JoyeBright/Multilingual_Multimodal_Universal_MT/blob/main/DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Data Shuffling**

In [1]:
path = "/content/drive/MyDrive/Multilingual_Multimodal_Universal_MT/Per_Eng_Trans_V1/raw_data/TEP"

In [2]:
samples = 90000

In [3]:
import numpy as np

In [4]:
!pip3 install torch torchvision

In [5]:
train_src = path+"/train.tok.clean.src"

In [6]:
!head -10 $train_src

یون سنگ، تو میری از او بپرسی چیزی پیدا کرده یا نه؟
چيز غيرعادي اي نبود .
...البته کاملا مزه گوشت گاو را نداره
شماها باید به این خاتمه بدین .
بله. اما او به نظر آشفته بود. مشکلی پیش اومده؟
ولي خوني شده .
شيطنت کردن .
اما اگر عميقتر ببينيد برتر از اين هم وجود داره، يعني زير زمين
معنی سادش اینه که شما میخواید فرار کنید .
باقيماندن .


In [7]:
with open(train_src) as f:
    content = f.readlines()
    content = [x.strip() for x in content]

In [8]:
sentences = content

In [9]:
print(sentences[:10])

['یون سنگ، تو میری از او بپرسی چیزی پیدا کرده یا نه؟', 'چيز غيرعادي اي نبود .', '...البته کاملا مزه گوشت گاو را نداره', 'شماها باید به این خاتمه بدین .', 'بله. اما او به نظر آشفته بود. مشکلی پیش اومده؟', 'ولي خوني شده .', 'شيطنت کردن .', 'اما اگر عميقتر ببينيد برتر از اين هم وجود داره، يعني زير زمين', 'معنی سادش اینه که شما میخواید فرار کنید .', 'باقيماندن .']


In [10]:
from random import shuffle
shuffle(sentences)

In [11]:
print(sentences[:10])

['البته اون دزديده شده .', 'احمق نشو . کي ميتونه تو يک زمان تو دو تا کلاس باشه .', 'اينقدر بي احتياط نباش', 'برو بالا و دوباره سعي كن .', 'من نمي تونم قبول كنم كه سربازهاي ارتشاز سربازهاي شخصي شكست بخورند', 'اون ميخواد با من صحبت کنه. بذاريد برم تو', 'من ازت خوشم مياد0 منو تو کارهاي بزرگي با هم خواهيم کرد .', 'تاثير گذارترين فرد در زندگي من .', 'بذار كمكت كنم نه ، دستت درد نكنه .', 'خانم ديسي فولر . يک لحظه صبر کنين .']


In [12]:
sentences = sentences [:samples]

## **Sentence-BERT**

In [17]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.1572, 0.0051, 0.4584],
        [0.6009, 0.6217, 0.1572],
        [0.0671, 0.9708, 0.8931],
        [0.5800, 0.9976, 0.8607],
        [0.4220, 0.2991, 0.1189]])


In [13]:
!pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.4.1.2)


In [ ]:
from sentence_transformers import SentenceTransformer
name = 'https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/v0.2/xlm-r-100langs-bert-base-nli-mean-tokens.zip'

In [ ]:
model = SentenceTransformer(model_name_or_path=name, device='cuda')

In [ ]:
print("Max Sequence Length:", model.max_seq_length)

Max Sequence Length: 128


In [ ]:
sentence_embeddings = model.encode(sentences=sentences, show_progress_bar=True, convert_to_numpy=True)

In [ ]:
print(type(sentence_embeddings))
print(sentence_embeddings.shape)

<class 'numpy.ndarray'>
(610086, 768)


In [19]:
import pickle

In [ ]:
#Store sentences & embeddings on disc
with open('embeddings.pkl', "wb") as fOut:
    pickle.dump({'sentences': sentences, 'embeddings': sentence_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [28]:
#Load sentences & embeddings from disc
with open('/content/drive/MyDrive/Multilingual_Multimodal_Universal_MT/Domain_Adaptation/SentenceBERT/embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data['sentences']
    stored_embeddings = stored_data['embeddings']

In [29]:
print(stored_embeddings.shape)
print(type(stored_embeddings))

(610086, 768)
<class 'numpy.ndarray'>


In [30]:
stored_embeddings = stored_embeddings[:samples]
stored_sentences = stored_sentences[:samples]

print(stored_embeddings.shape)

(40000, 768)


In [31]:
print(stored_embeddings[:1])

[[ 0.4593516  -0.01457303  1.7394984   0.4404026   0.6362638   0.2528962
   0.11195075  0.2921515   0.10742761  0.15947405 -0.21293342  0.5458668
   0.2740503   0.5479267  -0.32321924  0.13399549  0.04454777 -0.57006615
   0.18741901 -0.5034281   0.17253752  0.3391332   0.10633039 -0.69341135
   0.30869687 -0.06997975 -0.12753329 -0.82400626 -0.6240134   0.20505981
   0.03349856 -0.04983833 -0.27879694 -0.17779048 -0.898158    0.9478081
   0.7969705   0.10409191  0.04230267  0.0060135   0.0042068   0.06179347
   0.30250612  0.8499539  -0.9323933  -0.5255916   0.36498743  0.12332773
  -0.19779778 -1.1141446  -0.5156098   0.08481535  0.55320185  0.39266366
  -0.669689   -0.03589565  0.5764537  -1.7097796  -0.14665571  0.27364424
  -1.2197844  -0.17900002  0.81105083  0.49735194 -0.09490854  0.6097086
   0.13635202 -0.41592193 -0.5177906  -0.04319844  0.6383545  -0.5826326
  -0.08961103  0.6948251  -0.37232247 -0.10304417  0.00706872  0.81251115
   0.48310867  0.6950191  -0.22152555  0.08

In [ ]:
!pip install gpustat

In [ ]:
!gpustat -cp

57e15a3dbd9d         Sat Feb  6 15:23:44 2021  418.67
[0] Tesla T4         | 61'C,   0 % |     0 / 15079 MB |


In [24]:
from sentence_transformers import util

## **Clustering using Sentence-Transformers package**

In [32]:
def community_detection(embeddings, threshold=0.75, min_community_size=10, init_max_size=1000):
    """
    Function for Fast Community Detection
    Finds in the embeddings all communities, i.e. embeddings that are close (closer than threshold).
    Returns only communities that are larger than min_community_size. The communities are returned
    in decreasing order. The first element in each list is the central point in the community.
    """

    # Compute cosine similarity scores
    cos_scores = util.pytorch_cos_sim(embeddings, embeddings)

    # Minimum size for a community
    top_k_values, _ = cos_scores.topk(k=min_community_size, largest=True)

    # Filter for rows >= min_threshold
    extracted_communities = []
    for i in range(len(top_k_values)):
        if top_k_values[i][-1] >= threshold:
            new_cluster = []

            # Only check top k most similar entries
            top_val_large, top_idx_large = cos_scores[i].topk(k=init_max_size, largest=True)
            top_idx_large = top_idx_large.tolist()
            top_val_large = top_val_large.tolist()

            if top_val_large[-1] < threshold:
                for idx, val in zip(top_idx_large, top_val_large):
                    if val < threshold:
                        break

                    new_cluster.append(idx)
            else:
                # Iterate over all entries (slow)
                for idx, val in enumerate(cos_scores[i].tolist()):
                    if val >= threshold:
                        new_cluster.append(idx)

            extracted_communities.append(new_cluster)

    # Largest cluster first
    extracted_communities = sorted(extracted_communities, key=lambda x: len(x), reverse=True)

    # Step 2) Remove overlapping communities
    unique_communities = []
    extracted_ids = set()

    for community in extracted_communities:
        add_cluster = True
        for idx in community:
            if idx in extracted_ids:
                add_cluster = False
                break

        if add_cluster:
            unique_communities.append(community)
            for idx in community:
                extracted_ids.add(idx)

    return unique_communities

In [33]:
import time
print("Start clustering")
start_time = time.time()
clusters = community_detection(stored_embeddings, min_community_size=40, threshold=0.95)
#Print all cluster / communities
for i, cluster in enumerate(clusters):
    print("\nCluster {}, #{} Elements ".format(i+1, len(cluster)))
    for sentence_id in cluster:
        print("\t", stored_sentences[sentence_id])

print("Clustering done after {:.2f} sec".format(time.time() - start_time))

Start clustering

Cluster 1, #1464 Elements 
	 ... هايوسو
	 چي اسلووکي .
	 زبر تن .
	 کيفم .
	 ميخ محور .
	 منسوب .
	 یانی یانی .
	 بله لطفا از اين طرف
	 زمره .
	 ايان بسه
	 اف .
	 بارو .
	 بنويس .
	 سراج .
	 در پهلو .
	 مته دستي .
	 افتراء .
	 ياور .
	 باجداري .
	 ترسيدي .
	 زيان بانگ بله .
	 اين يکي چي .
	 باريدن .
	 دستگيرش کنيد
	 استادوار .
	 مخل .
	 تکميل کردن .
	 خب ديگه .
	 ميز اتو .
	 مترادف .
	 تاب آوردني .
	 الفي . الفي .
	 کف بيني .
	 جاپا .
	 بيختن .
	 سفرس .
	 . آرزت
	 ميزني .
	 ,,, ولي
	 قره کل .
	 - مي فهيمي؟ - بله.
	 حق دادن .
	 بيحركت .
	 گوري .
	 خب حالا كجان .
	 ترديد .
	 پس اون کسي که ميخواست...
	 خودت را ببين .
	 بي .
	 که در آن .
	 توزين کردن .
	 کک گزيدگي .
	 مقاربتي .
	 طوطي وار .
	 بنويسيد .
	 زي .
	 پوستک .
	 قلمرواسقفي .
	 دريع داشتن .
	 پس ديد .
	 من ميگم ، ما .
	 دائه سو!
	 کلوين اينمن
	 اداي سخن .
	 ... رييس
	 کورت .
	 عفت .
	 بي فصل .
	 سرودني .
	 خودفرضي .
	 نيو .
	 لفاظي .
	 باند .
	 چرا,
	 بيليون .
	 مي كشمت
	 مکتشف .
	 حالا بگو
	 ايا ميدونيد
	 آبگونگي

## **Top2Vec**

In [ ]:
!pip install top2vec[sentence_transformers]

     |████████████████████████████████| 307kB 18.7MB/s 
     |████████████████████████████████| 6.4MB 23.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp36-cp36m-linux_x86_64.whl size=2311380 sha256=7a0ff6ced384420af60da0b51886742e83c5c9a2ed0f9c43eb9361dcd3e4246d
  Stored in directory: /root/.cache/pip/wheels/42/63/fb/314ad6c3b270887a3ecb588b8e5aac50b0fad38ff89bb6dff2
Successfully built hdbscan
ERROR: hdbscan 0.8.27 has requirement joblib>=1.0, but you'll have joblib 0.17.0 which is incompatible.
ERROR: top2vec 1.0.21 has requirement numpy==1.19.2, but you'll have numpy 1.19.5 which is incompatible.
  Found existing installation: joblib 1.0.0
    Uninstalling joblib-1.0.0:
      Successfully uninstalled joblib-1.0.0


In [ ]:
from top2vec import Top2Vec

In [ ]:
model = Top2Vec(stored_sentences[:samples], embedding_model='distiluse-base-multilingual-cased')

2021-02-06 15:25:25,802 - top2vec - INFO - Pre-processing documents for training
2021-02-06 15:25:26,095 - top2vec - INFO - Downloading distiluse-base-multilingual-cased model
2021-02-06 15:25:43,620 - top2vec - INFO - Creating joint document/word embedding
2021-02-06 15:25:55,677 - top2vec - INFO - Creating lower dimension embedding of documents
2021-02-06 15:26:34,241 - top2vec - INFO - Finding dense areas of documents
2021-02-06 15:26:34,688 - top2vec - INFO - Finding topics


In [ ]:
print("Number of topics:", model.get_num_topics())


Number of topics: 96


In [ ]:
topic_sizes, topic_nums = model.get_topic_sizes()
print("topic size", topic_sizes)
print("topic number", topic_nums)

topic size [803 390 315 264 260 223 211 201 199 196 195 191 171 165 161 157 143 142
 141 141 133 130 129 122 118 116 116 113 108 105 105 105 101  98  95  92
  91  91  90  88  87  86  85  85  85  84  83  81  80  79  78  75  74  73
  73  73  66  65  65  64  64  63  62  62  62  61  59  58  56  54  53  53
  52  52  48  48  48  44  44  42  42  42  42  41  41  39  38  36  36  33
  31  30  30  28  27  23]
topic number [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95]


In [ ]:
topic_words, word_scores, topic_nums = model.get_topics(3)

In [ ]:
print(topic_words)

[['شد' 'پيش' 'کردن' 'مورد' 'به' 'می' 'ولي' 'روي' 'را' 'توي' 'هر' 'كن'
  'خاطر' 'يک' 'شده' 'از' 'کار' 'يه' 'بيرون' 'كار' 'بهش' 'رو' 'انجام'
  'حرف' 'اي' 'يك' 'هستند' 'او' 'کي' 'دست' 'در' 'سر' 'يا' 'اقاي' 'کرد'
  'بده' 'راه' 'مي' 'ما' 'دوست' 'كه' 'کن' 'هم' 'که' 'کني' 'بوده' 'منو'
  'کنيد' 'اما' 'نفر']
 ['کنم' 'ميکنم' 'ميخوام' 'کردم' 'كنم' 'دارم' 'هستم' 'ميتونم' 'ميشه'
  'لطفا' 'بودم' 'بي' 'بهت' 'بده' 'با' 'منو' 'کنيم' 'گفتم' 'بر' 'بيا'
  'شده' 'برو' 'مي' 'ميخواي' 'کني' 'داشته' 'بهش' 'کن' 'ميکني' 'اونو' 'يا'
  'بوده' 'کنيد' 'تو' 'روي' 'فکر' 'يك' 'توي' 'خاطر' 'باش' 'رو' 'تمام'
  'هستي' 'اقاي' 'يک' 'اوه' 'به' 'شما' 'بيرون' 'می']
 ['ميخواي' 'شده' 'اونا' 'اونها' 'هستند' 'روي' 'بده' 'اوه' 'ها' 'بر'
  'اقاي' 'هم' 'ميدوني' 'برو' 'کردن' 'لطفا' 'کنيد' 'توي' 'هاي' 'کرد'
  'کرده' 'سر' 'ميکني' 'بچه' 'کني' 'با' 'بهت' 'بيا' 'يا' 'بهش' 'براي'
  'ولي' 'کي' 'يک' 'اينجا' 'پيش' 'دست' 'خب' 'صبر' 'هر' 'مرد' 'سلام' 'كه'
  'اونجا' 'کنيم' 'نگاه' 'شما' 'به' 'می' 'شد']]


In [ ]:
words, word_scores = model.similar_words(keywords=["خاطر"], keywords_neg=[], num_words=20)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

شد 0.8685322168532293
به 0.8597604432911965
را 0.849143525509674
هر 0.8305449815171645
می 0.8293873189723151
پيش 0.8251956479906721
كن 0.8215255084392645
ولي 0.8194528472663456
مورد 0.8136457654968112
رو 0.812031051290705
او 0.8035998871277281
بهش 0.799646851987762
کردن 0.7995370442710003
يك 0.7971300720991035
انجام 0.7921365155972069
روي 0.7919249168401934
شده 0.7868035389615263
يک 0.7866085826817532
مي 0.7851309341018387
توي 0.7813211966054003


# **BERTopic**

In [14]:
!pip install bertopic

In [15]:
!pip install bertopic[visualization]

In [16]:
from bertopic import BERTopic

In [17]:
model = BERTopic(language="persian", verbose=True, min_topic_size = 200, calculate_probabilities = False)

In [18]:
print(sentences[:10])

['البته اون دزديده شده .', 'احمق نشو . کي ميتونه تو يک زمان تو دو تا کلاس باشه .', 'اينقدر بي احتياط نباش', 'برو بالا و دوباره سعي كن .', 'من نمي تونم قبول كنم كه سربازهاي ارتشاز سربازهاي شخصي شكست بخورند', 'اون ميخواد با من صحبت کنه. بذاريد برم تو', 'من ازت خوشم مياد0 منو تو کارهاي بزرگي با هم خواهيم کرد .', 'تاثير گذارترين فرد در زندگي من .', 'بذار كمكت كنم نه ، دستت درد نكنه .', 'خانم ديسي فولر . يک لحظه صبر کنين .']


In [19]:
topics, probabilities = model.fit_transform(sentences)

2021-02-07 05:42:57,587 - BERTopic - Loaded embedding model
2021-02-07 05:44:35,527 - BERTopic - Transformed documents to Embeddings
2021-02-07 05:46:42,121 - BERTopic - Reduced dimensionality with UMAP
2021-02-07 05:47:02,827 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2021-02-07 05:47:20,833 - BERTopic - Loaded embedding model
2021-02-07 05:47:21,433 - BERTopic - Transformed documents to Embeddings


In [20]:
model.save("/content/drive/MyDrive/Multilingual_Multimodal_Universal_MT/Domain_Adaptation/BERTopic/BERTopic-shuffle3-90000")

/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [43]:
model = BERTopic.load("/content/drive/MyDrive/Multilingual_Multimodal_Universal_MT/Domain_Adaptation/BERTopic/BERTopic-shuffle1-90000")

### **Results from shuffle 1**

In [44]:
print(model.get_topic_freq().head(8))

   Topic  Count
0     -1  37712
1     21  10690
2     31   4378
3     44   4249
4     38   3815
5     24   3593
6      7   3566
7     23   2105


In [45]:
p = model.get_topic_freq().head(8)

In [46]:
for index, Topic in p.iterrows():
  print("Topic ID:", Topic[0], ", Count", Topic[1])
  print(np.asarray(model.get_topic(Topic[0])))
  print("")

Topic ID: -1 , Count 37712
[['تو' '0.03458628919583673']
 ['در' '0.02634422407954865']
 ['با' '0.024433633008791544']
 ['بايد' '0.02014700588564294']
 ['شما' '0.018612101957461262']
 ['بود' '0.016889716976383386']
 ['باشه' '0.015615369200083901']
 ['خيلي' '0.014300552781332706']
 ['خوب' '0.01400150070890715']
 ['همه' '0.013446245148449264']]

Topic ID: 21 , Count 10690
[['نه' '0.08129892886018769']
 ['نيست' '0.05755828151424445']
 ['هيچ' '0.04321033519050902']
 ['نمي' '0.038046017707403096']
 ['نداره' '0.022465812537815634']
 ['ندارم' '0.01804373536575255']
 ['هرگز' '0.017795613518963718']
 ['نميتونم' '0.01758312765275363']
 ['نميدونم' '0.01668754647137372']
 ['نبايد' '0.01657514641995119']]

Topic ID: 31 , Count 4378
[['در' '0.02355647490741937']
 ['زدن' '0.018799857469401226']
 ['کننده' '0.016378599277208805']
 ['سازي' '0.01615490425253941']
 ['با' '0.01294032144070021']
 ['خانم' '0.011531375985070771']
 ['وايسا' '0.01143401131157352']
 ['اهل' '0.01064951025651073']
 ['دهنده' '0.0102

In [47]:
model.visualize_topics()

### **Results from shuffle 2**

In [36]:
print(model.get_topic_freq().head(8))

   Topic  Count
0     -1  35757
1     19  11078
2     37   5099
3     39   4694
4     44   4543
5      7   3974
6     30   3876
7     26   2109


In [37]:
p = model.get_topic_freq().head(8)

In [42]:
for index, Topic in p.iterrows():
  print("Topic ID:", Topic[0], ", Count", Topic[1])
  print(np.asarray(model.get_topic(Topic[0])))
  print("")

Topic ID: -1 , Count 35757
[['تو' '0.036136357526353496']
 ['در' '0.02769528636195063']
 ['با' '0.023561724396440907']
 ['بايد' '0.019677344296791764']
 ['شما' '0.019615303319851764']
 ['باشه' '0.016098261440546338']
 ['بود' '0.016062790021225203']
 ['خيلي' '0.015229564115238287']
 ['همه' '0.012776314563407413']
 ['اينجا' '0.012542849957497736']]

Topic ID: 19 , Count 11078
[['نه' '0.07872530091876091']
 ['نيست' '0.056816231729363335']
 ['هيچ' '0.04523769524343889']
 ['نمي' '0.03612516386164729']
 ['ديگه' '0.023937826980021774']
 ['ما' '0.0235677317425978']
 ['نداره' '0.02132030556354764']
 ['نميتونم' '0.01948428927592129']
 ['ندارم' '0.017888807443109892']
 ['نميدونم' '0.01651422034277171']]

Topic ID: 37 , Count 5099
[['کردن' '0.08331857516503238']
 ['در' '0.019558309854126744']
 ['زدن' '0.017827434818262503']
 ['وايسا' '0.017699132237280098']
 ['خانم' '0.01742663268398777']
 ['اسم' '0.016654762869056493']
 ['کننده' '0.014275625325363943']
 ['سازي' '0.012449176053466933']
 ['يا' '0.0

In [40]:
model.visualize_topics()

### **Results from shuffle 3**

In [21]:
print(model.get_topic_freq().head(8))

   Topic  Count
0     -1  39547
1     26   9909
2     45   5000
3      7   3993
4     43   3855
5     29   3506
6     24   2212
7     23   1583


In [22]:
p = model.get_topic_freq().head(8)

In [23]:
for index, Topic in p.iterrows():
  print("Topic ID:", Topic[0], ", Count", Topic[1])
  print(np.asarray(model.get_topic(Topic[0])))
  print("")

Topic ID: -1 , Count 39547
[['تو' '0.03453889954651894']
 ['در' '0.02674938415027857']
 ['با' '0.024976809264135424']
 ['بايد' '0.019174573535836348']
 ['شما' '0.017104459234992306']
 ['باشه' '0.01526726079441414']
 ['خيلي' '0.014553911456499171']
 ['خوب' '0.013473308396016072']
 ['اگر' '0.012591130530547561']
 ['اما' '0.012260698205163565']]

Topic ID: 26 , Count 9909
[['نه' '0.08004488852443642']
 ['نيست' '0.05858433996508915']
 ['هيچ' '0.04864951775618506']
 ['نمي' '0.03766583483527043']
 ['نداره' '0.022674556447290984']
 ['ديگه' '0.021419719054490263']
 ['نميتونم' '0.018813642542073274']
 ['ندارم' '0.018052049632865778']
 ['نبايد' '0.01684738855934133']
 ['نيستم' '0.01666824823054557']]

Topic ID: 45 , Count 5000
[['من' '0.07158784200559841']
 ['همه' '0.0394947081386051']
 ['بله' '0.03510159557976668']
 ['بايد' '0.03221886452724462']
 ['آره' '0.02745034179251883']
 ['ميدونم' '0.02226481152859184']
 ['دارم' '0.021274421993458238']
 ['کنم' '0.019772835842710067']
 ['کردم' '0.01827885

In [24]:
model.visualize_topics()